In [1]:
import numpy as np
import pandas as pd
import os
import copy
import warnings
warnings.filterwarnings('ignore')

from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
pd.set_option('display.max_columns', None)

In [3]:
trainPath = '../user_data/tmp_data/'
pick = pd.read_csv(trainPath+'pickAll.txt')
delivery = pd.read_csv(trainPath+'deliveryAll.txt')

In [8]:
# 根据选单模型生成每个订单动作后续取单最有可能取的单和送单最有可能送的单，得到下一步的决策表
def getDecisionData(pickupOrder,deliveryOrder):
    
    pick = pickupOrder[['订单号','动作','动作时间','通知时间','可以取单时间','承诺送达时间','骑手上个动作时间','骑手经度','骑手等级','骑手纬度','骑手距离','骑手速度','最大负荷量']]
    delivery = deliveryOrder[['订单号','动作','动作时间','通知时间','可以取单时间','承诺送达时间','骑手上个动作时间','骑手经度','骑手等级','骑手纬度','骑手距离','骑手速度','最大负荷量']]
     
    dataSort = pd.concat([pick,delivery])
    dataSort.drop_duplicates(inplace =True)
    dataSort['理论完成时间'] = (dataSort['骑手距离']/dataSort['骑手速度'] +dataSort['骑手上个动作时间']).apply(int)
    
    columnsName =['取单号','商家经度','商家纬度','商家距离','实际取单时间','理论取单时间','取单分配时间','取单可取时间','取单承诺送达时间','取单天气','送单号','客户经度','客户纬度','客户距离','实际送单时间','理论送单时间','送单分配时间','送单可取时间','送单承诺送达时间','送单天气','当前已取单数']
    for st in columnsName:
        dataSort.loc[:,st] =0

    indexlist = list(range(len(dataSort)))
    dataSort.index = indexlist
    # 模型读取
    path = '../user_data/model_data/choose_model/'
    modelPick2lin =joblib.load(path+'RFpick2.pkl') # 双订单不同店取单模型
    modelPickSame2lin = joblib.load(path+'RFpickSame2.pkl') # 双订单同店取单模型
    
    modelPick3lin = joblib.load(path+'RFpick3.pkl') # 三订单不同店取单模型
    modelPick3PartSamelin = joblib.load(path+'RFpickpartSame3.pkl') # 三订单部分同店取单模型
    modelPick3Samelin = joblib.load(path+'RFpickSame3.pkl') # 三订单都同店取单模型
    
    modelDe2lin = joblib.load(path+'RFDE2.pkl') # 双订单不同店送单模型
    modelDeSame2lin = joblib.load(path+'RFDESame2.pkl') # 双订单同店送单模型

    modelDe3lin = joblib.load(path+'RFDE3.pkl')  # 三订单不同店送单
    modelDePartSame3lin = joblib.load(path+'RFDEpartSame3.pkl') # 三订单部分同店送单
    # 计算得到表
    for i in range(len(dataSort)):
        orderID = dataSort.loc[i,'订单号']
        
        #peopleID = dataSort.loc[i,'骑手ID']
        orderAction = dataSort.loc[i,'动作']
 
        order_assign = pickupOrder.loc[(pickupOrder['订单号'] ==orderID) &(pickupOrder['动作'] == orderAction),:]  #存放已发布，未提取
        order_pickup = deliveryOrder.loc[(deliveryOrder['订单号'] ==orderID) &(deliveryOrder['动作'] == orderAction),:]
       
       
        # 取单
        index=0
        if len(order_assign) ==1:
            index =0
        elif len(order_assign)==2:
            dis = order_assign.iloc[0,18] - order_assign.iloc[1,18]
            time1 = order_assign.iloc[0,19] - order_assign.iloc[1,19] # 理论时间差
            time2 = order_assign.iloc[0,20] - order_assign.iloc[1,20] # 通知时间差
            time3 = order_assign.iloc[0,21] - order_assign.iloc[1,21] # 可取时间差
            time4 = order_assign.iloc[0,22] - order_assign.iloc[1,22] # 承诺时间差
            if abs(dis)<0.001:
                temp = [[time2,time3,time4]]
                input1 = np.array(temp)
                index = int(modelPickSame2lin.predict(input1)[0])
            elif (dis>1500) |(dis<-1500):
                order_assign.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
                index=0
            else:
                temp = [[time1,time2,time3,time4]]
                input1 = np.array(temp)
                index = int(modelPick2lin.predict(input1)[0])
        elif len(order_assign) ==3:
            dis1 = order_assign.iloc[0,18] - order_assign.iloc[1,18]
            dis2 = order_assign.iloc[0,18] - order_assign.iloc[2,18]
            
            time4 = order_assign.iloc[0,20] - order_assign.iloc[1,20] # 1-2通知时间差
            time5 = order_assign.iloc[0,20] - order_assign.iloc[2,20] # 1-3通知时间差
            time6 = order_assign.iloc[1,20] - order_assign.iloc[2,20] # 2-3通知时间差

            time7 = order_assign.iloc[0,21] - order_assign.iloc[1,21] # 1-2可取时间差
            time8 = order_assign.iloc[0,21] - order_assign.iloc[2,21] # 1-3可取时间差
            time9 = order_assign.iloc[1,21] - order_assign.iloc[2,21] # 2-3可取时间差

            time10 = order_assign.iloc[0,22] - order_assign.iloc[1,22] # 1-2承诺时间差
            time11 = order_assign.iloc[0,22] - order_assign.iloc[2,22] # 1-3承诺时间差
            time12 = order_assign.iloc[1,22] - order_assign.iloc[2,22] # 2-3承诺时间差

            if (abs(dis1)<0.001) &(abs(dis2)<0.001): # 三订单同一家店 
                temp = [[time4,time5,time6,time7,time8,time9,time10,time11,time12]]
                input1 = np.array(temp)
                index = int(modelPick3Samelin.predict(input1)[0])
            elif (abs(dis1) >0.1) &(abs(dis2)>0.1): # 三订单不同店
                time1 = order_assign.iloc[0,19] - order_assign.iloc[1,19] # 1-2理论时间差
                time2 = order_assign.iloc[0,19] - order_assign.iloc[2,19] # 1-3理论时间差
                time3 = order_assign.iloc[1,19] - order_assign.iloc[2,19] # 2-3理论时间差
                temp = [[time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,time11,time12]]
                input1 = np.array(temp)
                index = int(modelPick3lin.predict(input1)[0])
            else:
                time1 = order_assign.iloc[0,19] - order_assign.iloc[1,19] # 1-2理论时间差
                time2 = order_assign.iloc[0,19] - order_assign.iloc[2,19] # 1-3理论时间差
                time3 = order_assign.iloc[1,19] - order_assign.iloc[2,19] # 2-3理论时间差
                temp = [[time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,time11,time12]]
                input1 = np.array(temp)
                index = int(modelPick3PartSamelin.predict(input1)[0])
        else:              # 四个单以上
            order_assign.sort_values(by=['距离','订单可取时间'],inplace =True)
            index = 0
            
        # 送单
        index1 =0
        if len(order_pickup) ==1:
            index1 =0
        elif len(order_pickup) ==2:
            dis = order_pickup.iloc[0,18] - order_pickup.iloc[1,18]
            time1 = order_pickup.iloc[0,19] - order_pickup.iloc[1,19] # 理论时间差
            time2 = order_pickup.iloc[0,20] - order_pickup.iloc[1,20] # 通知时间差
            time3 = order_pickup.iloc[0,21] - order_pickup.iloc[1,21] # 可取时间差
            time4 = order_pickup.iloc[0,22] - order_pickup.iloc[1,22] # 承诺时间差
            if abs(dis)<0.001: # 同单
                temp = [[time2,time4]]
                input1 = np.array(temp)
                index1 = int(modelDeSame2lin.predict(input1)[0])
            elif (dis>1000) |(dis<-1000):
                order_pickup.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
                index1=0
            else:
                temp = [[time1,time2,time3,time4]]
                input1 = np.array(temp)
                index1 = int(modelDe2lin.predict(input1)[0])
        elif len(order_pickup) ==3:
            dis1 = order_pickup.iloc[0,18] - order_pickup.iloc[1,18]
            dis2 = order_pickup.iloc[0,18] - order_pickup.iloc[2,18]
            
            time1 = order_pickup.iloc[0,19] - order_pickup.iloc[1,19] # 1-2理论时间差
            time2 = order_pickup.iloc[0,19] - order_pickup.iloc[2,19] # 1-3理论时间差
            time3 = order_pickup.iloc[1,19] - order_pickup.iloc[2,19] # 2-3理论时间差
            
            time10 = order_pickup.iloc[0,22] - order_pickup.iloc[1,22] # 1-2承诺时间差
            time11 = order_pickup.iloc[0,22] - order_pickup.iloc[2,22] # 1-3承诺时间差
            time12 = order_pickup.iloc[1,22] - order_pickup.iloc[2,22] # 2-3承诺时间差

            if (abs(dis1)<0.001) &(abs(dis2)<0.001): # 三订单同一家店
                order_pickup.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
                index1 =0
            elif (abs(dis1) >0.1) &(abs(dis2)>0.1): # 三订单不同店
                temp = [[time1,time2,time3,time10,time11,time12]]
                input1 = np.array(temp)
                index1 = int(modelDe3lin.predict(input1)[0])
            else:
                time4 = order_pickup.iloc[0,20] - order_pickup.iloc[1,20] # 1-2通知时间差
                time5 = order_pickup.iloc[0,20] - order_pickup.iloc[2,20] # 1-3通知时间差
                time6 = order_pickup.iloc[1,20] - order_pickup.iloc[2,20] # 2-3通知时间差

                time7 = order_pickup.iloc[0,21] - order_pickup.iloc[1,21] # 1-2可取时间差
                time8 = order_pickup.iloc[0,21] - order_pickup.iloc[2,21] # 1-3可取时间差
                time9 = order_pickup.iloc[1,21] - order_pickup.iloc[2,21] # 2-3可取时间差
                
                temp = [[time1,time2,time3,time4,time5,time6,time7,time8,time9,time10,time11,time12]]
                input1 = np.array(temp)
                index1 = int(modelDePartSame3lin.predict(input1)[0])
        else:
            order_pickup.sort_values(by=['距离','订单承诺送达时间'],inplace =True)
            index1 =0
        
        
        
        if len(order_assign)==0:
            pass
        else:

            dataSort.loc[i,'取单号'] = order_assign.iloc[index,13]
            dataSort.loc[i,'商家经度'] = order_assign.iloc[index,16]
            dataSort.loc[i,'商家纬度'] = order_assign.iloc[index,17]
            dataSort.loc[i,'商家距离'] = order_assign.iloc[index,18]
            dataSort.loc[i,'实际取单时间'] = order_assign.iloc[index,15]
            dataSort.loc[i,'理论取单时间'] = order_assign.iloc[index,19]          
            dataSort.loc[i,'取单分配时间'] = order_assign.iloc[index,20]
            dataSort.loc[i,'取单可取时间'] = order_assign.iloc[index,21]
            dataSort.loc[i,'取单承诺送达时间'] = order_assign.iloc[index,22]
            dataSort.loc[i,'取单天气'] = order_assign.iloc[index,23]
                                      

        if len(order_pickup) ==0:
            pass
        else:
            dataSort.loc[i,'送单号'] = order_pickup.iloc[index1,13]
            dataSort.loc[i,'客户经度'] = order_pickup.iloc[index1,16]
            dataSort.loc[i,'客户纬度'] = order_pickup.iloc[index1,17]
            dataSort.loc[i,'客户距离'] = order_pickup.iloc[index1,18]
            dataSort.loc[i,'实际送单时间'] = order_pickup.iloc[index1,15]
            dataSort.loc[i,'理论送单时间'] = order_pickup.iloc[index1,19]
            dataSort.loc[i,'送单分配时间'] = order_pickup.iloc[index1,20]
            dataSort.loc[i,'送单可取时间'] = order_pickup.iloc[index1,21]
            dataSort.loc[i,'送单承诺送达时间'] = order_pickup.iloc[index1,22]
            dataSort.loc[i,'送单天气'] = order_pickup.iloc[index1,23]
            
        dataSort.loc[i,'当前已取单数'] = len(order_pickup)
        
     # 调整列名
    data_end=dataSort.reindex(columns=['订单号','动作','动作时间','理论完成时间','通知时间','可以取单时间','承诺送达时间','取单号','商家经度','商家纬度','商家距离','实际取单时间','理论取单时间','取单分配时间','取单可取时间','取单承诺送达时间','取单天气','送单号','客户经度','客户纬度','客户距离','实际送单时间','理论送单时间','送单分配时间','送单可取时间','送单承诺送达时间','送单天气','骑手上个动作时间','当前已取单数','骑手经度','骑手纬度','骑手距离','骑手等级','骑手速度','最大负荷量'])
    
    # 计算一致性
    data_pick = data_end.loc[data_end['动作']=='PICKUP',:]
    data_delivery = data_end.loc[data_end['动作'] == 'DELIVERY',:]
    data_pick['一致性'] = (data_pick['订单号']-data_pick['取单号']).apply(lambda x :1 if x==0 else 0)
    data_delivery['一致性'] = (data_delivery['订单号']-data_delivery['送单号']).apply(lambda x: 1 if x==0 else 0)
    
    data_save = pd.concat([data_pick,data_delivery])
    

    return  data_save
    
    

In [9]:
data = getDecisionData(pick,delivery)

In [14]:
data.to_csv('../user_data/tmp_data/decisionData.txt',index=False,encoding='utf-8')